In [7]:
import kagglehub
path = kagglehub.dataset_download("andrewmvd/heart-failure-clinical-data")

print("Path to dataset files:", path)

100%|██████████| 3.97k/3.97k [00:00<00:00, 2.03MB/s]

Extracting files...
Path to dataset files: C:\Users\13710\.cache\kagglehub\datasets\andrewmvd\heart-failure-clinical-data\versions\1


In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load data
data = pd.read_csv('./data/heart_failure_clinical_records_dataset.csv')

# Preprocess data
features = data.drop(columns=['DEATH_EVENT'])
labels = data['DEATH_EVENT']

# Split data
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

C:\Users\13710\AppData\Roaming\Python\Python39\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\13710\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [15]:
import torch
import torch.nn as nn
import torch.optim as optim

class AttentionModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(AttentionModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.attention = nn.Linear(hidden_dim, 1)
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        attn_weights = torch.softmax(self.attention(lstm_out), dim=1)
        context_vector = torch.sum(attn_weights * lstm_out, dim=1)
        output = self.fc(context_vector)
        return output

# Hyperparameters
input_dim = X_train.shape[1]
hidden_dim = 64
output_dim = 1

# Model, loss function, optimizer
model = AttentionModel(input_dim, hidden_dim, output_dim)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [18]:
# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).unsqueeze(1)
if len(X_train_tensor.shape) == 2:
    X_train_tensor = X_train_tensor.unsqueeze(1)
if len(X_test_tensor.shape) == 2:
    X_test_tensor = X_test_tensor.unsqueeze(1)

# Training loop
num_epochs = 2000
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluation
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    test_loss = criterion(test_outputs, y_test_tensor)
    print(f'Test Loss: {test_loss.item():.4f}')

Epoch [100/2000], Loss: 0.3962
Epoch [200/2000], Loss: 0.2714
Epoch [300/2000], Loss: 0.1905
Epoch [400/2000], Loss: 0.1255
Epoch [500/2000], Loss: 0.0790
Epoch [600/2000], Loss: 0.0496
Epoch [700/2000], Loss: 0.0325
Epoch [800/2000], Loss: 0.0224
Epoch [900/2000], Loss: 0.0162
Epoch [1000/2000], Loss: 0.0122
Epoch [1100/2000], Loss: 0.0095
Epoch [1200/2000], Loss: 0.0076
Epoch [1300/2000], Loss: 0.0062
Epoch [1400/2000], Loss: 0.0051
Epoch [1500/2000], Loss: 0.0043
Epoch [1600/2000], Loss: 0.0037
Epoch [1700/2000], Loss: 0.0032
Epoch [1800/2000], Loss: 0.0028
Epoch [1900/2000], Loss: 0.0024
Epoch [2000/2000], Loss: 0.0021
Test Loss: 1.5244


In [19]:
def generate_recommendations(predictions):
    recommendations = []
    for pred in predictions:
        if pred > 0.5:
            recommendations.append("High risk of hypertension. Recommend lifestyle changes and regular monitoring.")
        else:
            recommendations.append("Low risk of hypertension. Maintain a healthy lifestyle.")
    return recommendations

# Generate predictions
with torch.no_grad():
    predictions = torch.sigmoid(model(X_test_tensor)).numpy()

# Generate recommendations
recommendations = generate_recommendations(predictions)
for i, rec in enumerate(recommendations[:5]):
    print(f'Patient {i+1}: {rec}')

Patient 1: Low risk of hypertension. Maintain a healthy lifestyle.
Patient 2: Low risk of hypertension. Maintain a healthy lifestyle.
Patient 3: Low risk of hypertension. Maintain a healthy lifestyle.
Patient 4: High risk of hypertension. Recommend lifestyle changes and regular monitoring.
Patient 5: Low risk of hypertension. Maintain a healthy lifestyle.
